> **Jupyter slideshow:** This notebook can be displayed as slides. To view it as a slideshow in your browser type in the console:


> `> jupyter nbconvert [this_notebook.ipynb] --to slides --post serve`


> To toggle off the slideshow cell formatting, click the `CellToolbar` button, then `View --> Cell Toolbar --> None`

<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Flask

_Author: Alex Combs (NYC) _

---

<a id="learning-objectives"></a>
### Learning Objectives

- Understand what Flask is and what a web backend does.
- Understand how values are passed between a website and the Flask backend.
- Create a simple website locally that allows the user to interact with a model.
- Practice adding new features to our website.

### Lesson Guide

- [Learning Objectives](#learning-objectives)
- [Introduction](#introduction)

  - [Part 1 - Hello, world](#part----hello-world)
  - [Part 2 - Arguments and styling](#part----arguments-and-styling)
  - [Part 3 - Add in machine learning](#part----add-in-machine-learning)
  - [Part 4 - Create and train a Model](#part----create-and-train-a-model)
  - [Part 5 - Make a simple API](#part----make-a-simple-api)
  - [Part 6 - Make a simple webform](#part----make-a-simple-webform)


- [Independent Practice](#independent-practice)
- [Examples](#examples)
- [Additional Resources](#additional-resources)


<a id="introduction"></a>
## Introduction
---

![flask logo](http://flask.pocoo.org/static/logo/flask.png)
Flask is a fast, lightweight way to connect your Python scripts to a server. It's a simple and robust framework that can do small tasks (create a microblog, stand up a simple API) or complex ones (Pinterest's API, create a twitter clone).

Flask is called a _microframework_ because it is minimalistic and does not impose an application structure on you. Flask allows you to "plug in" libraries of your choice to add additional functionality. "Heavier" frameworks such as Django typically come with their own libraries (e.g. for database access). This is often convenient, but it takes more time to learn and is not as flexible.

Let's jump in with a simple example. Then, we'll expand it to show what it can do with your models. Flask comes preinstalled with Anaconda. However, if you don't have it you can install it:

```bash
$ pip install flask
```

Let's discuss how Flask is used to render a website:

1. First, each URL is mapped to a function (called **routing**).

2. When the user visits the URL, the associated function is called.

3. The function returns a string of HTML, which is directly rendered by the browser.

That's it! 

> Using this way of thinking, when visiting a URL you are actually making a function call to a remote computer.

> The URL is the function signature (including parameters), and the function returns a value back to the browser -- in this case a string of HTML. In fact, this is why interfaces for retrieving data online are called "Web APIs" -- they are just function calls disguised as URLs!

<a id="part----hello-world"></a>
### Part 1 - Hello, world

Create a new file called `controller.py` in the `titanic_app` folder. Type in this code line by line. Make sure you understand what's going on!

```Python
import flask

#----- CONFIG -----#
app = flask.Flask(__name__) # initialise Flask app var
app.config['DEBUG'] = True

#----- ROUTES -----#
@app.route("/")
def hello():
    return "Hello World!"

#----- MAIN SENTINEL -----#
if __name__ == '__main__':
    app.run()
```

Three things happen here:
- initialize the app
- use built-in decorators to define what happens on a page
- launch the app

Note that `app` must be defined prior to using the decorators! 

You can also choose to define your own `HOST` and `PORT` parameters for your Flask application.
```Python

#----- MAIN SENTINEL -----#
if __name__ == '__main__':
    '''Connects to the server'''

    HOST = '127.0.0.1'
    PORT = '5000'

    app.run(HOST, PORT)
```

Now launch the file from your command line:

```bash
$ python controller.py
* Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```
Go to that URL to see your app running on your `localhost`.

Typically, we develop apps locally using this development server. When working, we then upload the files to a web host such as EC2.

<a id="part----arguments-and-styling"></a>
### Part 2 - Arguments and styling

Add the following route underneath the hello() function:

```Python

#-------- ROUTES -----------#

@app.route('/greet/<name>')
def greet(name):
    '''Say hello to your first parameter'''
    return "Hello, %s!" %name
```
Save and relaunch the app. Now navigate to `http://127.0.0.1:5000/greet/Roger`. Your function should respond to that input!

Since the `return` statement sends text to an HTML page, you can style our original `hello()` function with HTML tags:

```Python
#-------- ROUTES -----------#

@app.route("/")
def hello():
    return '''
    <body>
    <h2> Hello World! <h2>
    </body>
    '''
```
We can also call a function `flask.render_template("webpage.html")` to serve a static page. More on this later.

<a id="step----set-up-a-webform"></a>
### Part 3 - Set up a webform

Now that we have the basic Flask app deployed, let's now set up a page with a form that can accept user input.

We'll first create the page. Create a new subfolder `templates` inside `titanic_app`. Then, create a file `input_page.html` in `templates`:


Now, let's add a form
```HTML
<!DOCTYPE html>
<html>
<form action="make_it_happen" method="post">
  First name: <input type="text" name="myname"><br><br>
  Excitement Level: <input type="text" name="mylevel"><br>
  <input type="submit" value="Submit">
</form>
</html>
```


That created our input page. We now need to modify Flask to accept the input from this page. We'll open the `controller.py` file and append the following under the ROUTES header:

```python
#-------- ROUTES -----------#

@app.route('/input_page')
def send_form():
  return flask.render_template('input_page.html')


@app.route('/make_it_happen', methods=['POST'])
def say_hi():
  name = flask.request.form['myname']
  excitement_level = flask.request.form['mylevel']
  return flask.render_template('stuff_you_know.html', name=name, lvl=excitement_level)

```


If everything went well, you should see this in your browser:

<img src="images/browser-3.png"> 

We'll walk through what each piece of this does. The first piece of code under the imports sets our app as Flask and points it to where we will be serving our HTML templates.

Next we have a function that runs when the default route for our app is called ('/'). We are returning the page we just created.

The next function is called when the form is submitted and takes the payload of that form: the entered name and the level of excitement level. It then sends it to what is called a string template. A string template allows us to pass in values from our code that can then be manipulated outside of our code in the HTML. This allows for a separation of concerns. The view can be dynamically generated based on the values passed to it, rather than having to code all the conditionals in Python on the Flask backend.

We capture the values passed to us in the form. Notice that the names in the HTML are captured and stored into Python variables.

Next, we discuss how to return an output with those values.

<a id="step----return-values-to-the-flask-backend"></a>
### Part 4 - Return Values to the Flask Backend

The return of that last function is returning a page that utilizes a Jinja2 string template. We are passing our Python variables, 'name' and 'excitement_level' to our string template under the Jinga2 variable names 'name' and 'lvl'. We could have used any variable names for these. We can also use as many as we want. Note, that they don't have to just be an integer. The can be a string or a list of values or basically any other type.

Create a new `stuff_you_know.html` file in the `templates` folder. Paste in the following:
```HTML
<!DOCTYPE html>
<html>
<head>
</head>
<body>
<center>
	{{ name }}, your excitement level is {{ lvl }}.
	{% if  lvl|int <= 5  %}
	Shame.<br><br>
	<img src="https://media.giphy.com/media/c2YyNySJ1CbFC/200.gif">

	{% elif lvl|int > 5 and lvl|int < 50 %}
	Meh. Try harder.<br><br>
	<img src="https://media0.giphy.com/media/11lLn6lwsdicLK/200.gif">

	{% elif lvl|int > 9000 %}
	OVER 9000!!<br><br>
	<img src="http://i0.kym-cdn.com/photos/images/original/000/845/401/195.gif">

	{% else %}
	That's fantastic. You are a star.<br><br>
	<img src="https://media4.giphy.com/media/arm7b1hJrw1BS/200.gif">
	{% endif %}
</center>
</body>
</html>

```

Lots of code there. What's going on?

Our variables that we passed in using Flask were 'name' and 'lvl' if you remember. Notice that we are using those here. They are in the curly braces: `{{ name }}` and `{{ lvl }}`. These are called templates, since Jinja2 will replace these placeholders with the variable values upon rendering. Just putting the curly braced values prints out to our page the values we passed in.

Here we also employed some conditional logic. This is how string templating works. We can do a number of operations using Jinja2; we can do everything from formatting values to showing or hiding data based on its values. It is very flexible and prevents us from having to change our backend code for every scenario.

<a id="step----test-it"></a>
### Part 5 - Test it!

Now that we have all the pieces, go ahead and go to the page for our app in your browser.

`localhost:5000`

Fill in the form. Try different values like 2, 7, 10.

Notice how the output changes with those values.

What are some ways you could use this in your work?

<a id="part----add-in-machine-learning"></a>
### Part 6 - Add in machine learning
We can use Flask as a way to share and host our machine learning predictions.


<a id="part----create-and-train-a-model"></a>
### Part 7 - Create and train a Model

First, import the following modules in the py header file:
```Python
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
```

Load in the `titanic.csv` dataset and create a model on it:

```Python
#-------- MODEL -----------#

df = pd.read_csv('titanic.csv')
include = ['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Survived']

# Create dummies and drop NaNs
df['Sex'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1)
df = df[include].dropna()

X = df[['Pclass', 'Sex', 'Age', 'Fare', 'SibSp']]
y = df['Survived']

PREDICTOR = RandomForestClassifier(n_estimators=100).fit(X, y)
```

You could also train and test your model in a Jupyter notebook and pickle the fitted model. This would result in a much faster load time for your app. In that case, your code would look something like this:

```Python
with open('picked_model.pkl', 'r') as picklefile:
    PREDICTOR = pickle.load(picklefile)
```

<a id="part----make-a-simple-api"></a>
### Part 8 - Make a simple API

Here's the fun part. Now that we have a `PREDICTOR`, we need to get some values to make our predictions.

One way to do this is to get information from the **URL parameters**. These are the part of a URL that come after the `?` and are matched by key:value pairs. For example, if you navigate to:

```
http://localhost:5000/predict?pclass=1&sex=1&age=18&fare=500&sibsp=0
```

then Flask can retrieve that data for you. Let's write a route to do just that:

```Python
#-------- ROUTES -----------#

@app.route('/predict', methods=["GET"])
def predict():
    pclass = flask.request.args['pclass']
    sex = flask.request.args['sex']
    age = flask.request.args['age']
    fare = flask.request.args['fare']
    sibsp = flask.request.args['sibsp']

    item = np.array([pclass, sex, age, fare, sibsp]).reshape(1,-1)
    score = PREDICTOR.predict_proba(item)
    results = {'survival chances': score[0,1], 'death chances': score[0,0]}
    return flask.jsonify(results)
```

And... voila! Save the file. Launch your app. You now have a simple API for your model.  

Play with the parameters in the URL. You should get the predicted probability of death and survival.

<a id="part----make-a-simple-webform"></a>
### Part 9 - Make a simple webform for the titanic dataset

Well that was exciting. But it doesn't look very nice. Let's create a simple web form to read in the inputs.

Create a file `titanic_input.html` in `templates`:

```html
<!doctype html>
<html>
  <head>
    <title>Titanic Survivor-O-Matic</title>
  </head>
  <body>

    <form action="/result" method="POST">
      <p>Class <input type="number" name="pclass" /></p>
      <p>Sex <input type="number" name="sex" /></p>
      <p>Age <input type="number" name="age" /></p>
      <p>Fare <input type="number" name="fare" /></p>
      <p># of siblings <input type="text" name="sibsp" /></p>

      <p><input type="submit" value="submit" /></p>
    </form>

  </body>
</html>
```

The two most common HTTP methods are `GET` and `POST`. When your browser visits a URL, it sends a `GET` request. However, when you submit a web form, typically a `POST` request is sent. For a `POST` request, a URL is still accessed. However, the parameters are sent inside the request body instead of as part of the URL (as we saw earlier with the `GET` parameters). Sending parameters in the body allows you to send more data than would fit inside the URL. However, it prevents a user from bookmarking the exact URL call and "hides" the submitted data from the user. (It only hides it from your average user -- the submitted data is actually easy to see using your browser's developer tools.)

Luckily, Flask knows how to read `form` tags in an HTML file that have been `POST`'d to the server.

Add two new decorators in below your first one. Write all this text out, don't copy paste:

```Python

#----- ROUTES -----#

# This method takes input via an HTML page
@app.route('/titanic_input')
def page():
   flask.render_template("titanic_input.html")

@app.route('/titanic_result', methods=['POST', 'GET'])
def result():
    '''Gets prediction using the HTML form'''
    if flask.request.method == 'POST':

       inputs = flask.request.form

       pclass = inputs['pclass'][0]
       sex = inputs['sex'][0]
       age = inputs['age'][0]
       fare = inputs['fare'][0]
       sibsp = inputs['sibsp'][0]

       item = np.array([pclass, sex, age, fare, sibsp]).reshape(1, -1)
       score = PREDICTOR.predict_proba(item)
       results = {'survival chances': score[0,1], 'death chances': score[0,0]}
       return flask.jsonify(results)

```

Save, close, and relaunch the app. Go to `http://127.0.0.1:5000/titanic_input` and type in your inputs.

Both methods should still be there. You can either play with the URL parameters at `/predict` or enter them at `/titanic_input`

<a id="independent-practice"></a>
## Independent Practice
---

See if you can customize and play around with the app you just built. Try the following things:

- Create the model in jupyter / pycharm, pickle it, and import it into your flask app.  Play around with the features used, change the model (try logistic regression)
- Make the app look nicer by playing with the HTML.  Can you use dropdowns?  Buttons?
- See if you can return more values to the page, like the predicted category, or the model's parameters.
- Whenever you let the user provide inputs, you must always validate them. For example, can you enter values that make the web app crash or return invalid output? To prevent this, you must always validate your inputs -- can you add some input validation here? Can you show the user an error if some inputs are not valid?


<a id="examples"></a>
## Examples
---

Here are some examples of Flask apps in action. Fork and clone the apps you like so you can play with them and edit them on your local machine.

Two apps using scikit-learn:
- [Visualizing the Iris dataset using Flask and Angular JS](https://github.com/ColCarroll/flask_angular_example)
- [Using Neural Nets to recognize images](https://github.com/mdlai/digit_recognition)

More websites built in Flask:
- [The Flask Website itself!](http://flask.pocoo.org/)
- [A reddit clone](https://github.com/codelucas/flask_reddit)


<a id="additional-resources"></a>
## Additional Resources
---

- [The Flask Documentation](http://flask.pocoo.org/docs/0.11/)
- [A Flask tutorial to follow along with](https://github.com/miguelgrinberg/flask-pycon2014)
- [The Flask mega tutorial](http://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-ii-templates)
